# Julia objectives

This notebook demonstrates the application of pyPESTO to objective functions defined in [Julia](https://julialang.org/).

As demonstration example, we use an [SIR disease dynamcis model](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology). For simulation, we use [DifferentialEquations.jl](https://diffeq.sciml.ai/stable/).

The code consists of multiple functions in the file `model_julia/SIR.jl`, wrapped in the namespace of a module `SIR`. We first speficy the reaction network via `Catalyst` (this is optional, one can also directly start with the differential equations), and then translate them to an ordinary differential equation (ODE) model in `OrdinaryDiffEq`. Then, we create synthetic observed `data` with normal noise. After that, we define a quadratic cost function `fun` (the negative log-likelihood under an additive normal noise model). We use backwards automatic differentiation via `Zygote` to derive also the gradient `grad` (there exist various derivative calculation methods, including forward and adjoint, and continuous and discrete sensitivities, see [SciMLSensitivity.jl](https://sensitivity.sciml.ai/dev/)):

In [ ]:
from pypesto.objective.julia import display_source_ipython

display_source_ipython("model_julia/SIR.jl")

We make the cost function and gradient known to `JuliaObjective`. Importing module and dependencies, and initializing some operations, can take some time due to pre-processing.

In [ ]:
import pypesto
from pypesto.objective.julia import JuliaObjective

In [ ]:
%%time

obj = JuliaObjective(
    module="SIR",
    source_file="model_julia/SIR.jl",
    fun="fun",
    grad="grad",
)

That's it -- now we have an objective function that we can simply use in pyPESTO like any other. Internally, it delegates all calls to Julia and translates results.

## Comments

Before continuing with the workflow, some comments:

When calling a function for the first time, Julia performs some internal pre-processing. Subsequent function calls are much more efficient.

In [ ]:
import numpy as np

x = np.array([-4.0, -2.0])

%time print(obj.get_fval(x))
%time print(obj.get_fval(x))
%time print(obj.get_grad(x))
%time print(obj.get_grad(x))

The outputs are already numpy arrays (in custom Julia objectives, it needs to be made sure that return objects can be parsed to floats and numpy arrays in Python).

In [ ]:
type(obj.get_grad(x))

Here, we use backward automatic differentiation to calculate the gradient. We can verify its correctness via finite difference checks:

In [ ]:
from pypesto import FD

fd = FD(obj, grad=True)

fd.get_grad(x)

Further, we can use the `JuliaObjective.get()` function to directly access any variable in the Julia module:

In [ ]:
sol_true = np.asarray(obj.get("sol_true"))
data = obj.get("data")

import matplotlib.pyplot as plt

for i, label in zip(range(3), ["S", "I", "R"]):
    plt.plot(sol_true[i], color=f"C{i}", label=label)
    plt.plot(data[i], 'x', color=f"C{i}")
plt.legend();

## Inference problem

Let us define an inference problem by specifying parameter bounds. Note that we use a log10-transformation of parameters in `fun`.

In [ ]:
from pypesto import Problem

# parameter boundaries
lb, ub = [-5.0, -3.0], [-3.0, -1.0]

# estimation problem
problem = Problem(obj, lb=lb, ub=ub)

## Optimization

Let us perform an optimization:

In [ ]:
%%time

# optimize
from pypesto import optimize

result = optimize.minimize(problem)

The objective function evaluations are quite fast!

We can also use parallelization, by passing a `pypesto.engine.MultiProcessEngine` to the `minimize` function.

In [ ]:
%%time

from pypesto.engine import MultiProcessEngine

engine = MultiProcessEngine()
result = optimize.minimize(problem, engine=engine, n_starts=100)

In [ ]:
from pypesto import visualize

visualize.waterfall(result);

## Sampling

Last, let us perform sampling from the log-posterior with implicitly defined uniform prior via the parameter bounds:

In [ ]:
%%time

from pypesto import sample

sampler = sample.AdaptiveParallelTemperingSampler(
    internal_sampler=sample.AdaptiveMetropolisSampler(), n_chains=3
)

result = sample.sample(
    problem, n_samples=10000, sampler=sampler, result=result
)

In [ ]:
pypesto.sample.geweke_test(result)
visualize.sampling_parameter_traces(
    result, use_problem_bounds=False, size=(12, 5)
);

In [ ]:
visualize.sampling_scatter(result, size=[13, 6]);